<a href="https://colab.research.google.com/github/daucel/Proyecto1/blob/main/Task_2_4_Dash_Callbacks_Colab_FIXED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 2.4 — Dash Callbacks (Colab-Ready, External Mode)

This notebook builds a small Dash app that meets the rubric:
- **Callback 1** toggles the *input container* (shows **Year** dropdown for Yearly report or **Vehicle Type** dropdown for Recession report).
- **Callback 2** renders the graph in the *output container* based on selections.

**Note:** Uses external mode to avoid inline deprecation errors in Colab.

In [2]:
# Install dependencies in Colab
!pip -q install jupyter-dash plotly pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.4 MB/s eta 0:00:00


In [3]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Sample dataset
df = pd.DataFrame({
    'Year':  [2010, 2011, 2012, 2013, 2014, 2015, 2016],
    'Sales': [100, 120,  90,  150,  130,  170,  160],
    'Unemployment': [7.5, 7.8, 8.1, 7.0, 6.8, 6.5, 6.3],
    'Vehicle_Type': ['Sedan','SUV','Sedan','SUV','Truck','Sedan','SUV']
})

# Build app
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H2('Automobile Sales Statistics Dashboard', style={'textAlign':'center','color':'blue'}),

    html.Label('Select Report Type:'),
    dcc.Dropdown(
        id='report-type',
        options=[{'label':'Recession Report','value':'recession'},
                 {'label':'Yearly Report','value':'yearly'}],
        value='recession', clearable=False
    ),
    html.Br(),

    # Input container with both controls; we toggle visibility
    html.Div([
        html.Div([
            html.Label('Year'),
            dcc.Dropdown(
                id='year-dropdown',
                options=[{'label':str(y),'value':y} for y in sorted(df['Year'].unique())],
                value=int(df['Year'].min()), clearable=False
            )
        ], id='year-block', style={'display':'none'}),

        html.Div([
            html.Label('Vehicle Type'),
            dcc.Dropdown(
                id='vehicle-dropdown',
                options=[{'label':v,'value':v} for v in sorted(df['Vehicle_Type'].unique())],
                value='Sedan', clearable=False
            )
        ], id='vehicle-block', style={'display':'block'})
    ], id='input-container'),

    html.Br(),
    html.Div(id='output-container')
])

# Callback 1: Toggle which input shows
@app.callback(
    Output('year-block','style'),
    Output('vehicle-block','style'),
    Input('report-type','value')
)
def toggle_inputs(report_type):
    if report_type == 'yearly':
        return {'display':'block'}, {'display':'none'}
    return {'display':'none'}, {'display':'block'}

# Callback 2: Render graph in output container
@app.callback(
    Output('output-container','children'),
    Input('report-type','value'),
    Input('year-dropdown','value'),
    Input('vehicle-dropdown','value')
)
def update_output(report_type, year_value, vehicle_value):
    if report_type == 'yearly' and year_value is not None:
        dff = df[df['Year']==year_value]
        fig = px.bar(dff, x='Vehicle_Type', y='Sales', title=f'Yearly Report — {year_value}')
        return dcc.Graph(figure=fig)
    elif report_type == 'recession' and vehicle_value is not None:
        dff = df[df['Vehicle_Type']==vehicle_value]
        fig = px.line(dff, x='Year', y='Sales', title=f'Recession Report — {vehicle_value}')
        return dcc.Graph(figure=fig)
    return 'Select inputs to see the report.'

# Launch (external URL to avoid inline deprecation)
app.run_server(mode='external', debug=False)


/usr/local/lib/python3.12/dist-packages/dash/dash.py:634: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



AttributeError: 'super' object has no attribute 'run_server'